In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/srilanka-geo/dristrict_aditional_data.csv
/kaggle/input/srilanka-geo/ds_aditional_data.csv
/kaggle/input/srilanka-geo/Grama_geo.json
/kaggle/input/srilanka-geo/District_geo.json
/kaggle/input/srilanka-geo/grama_niladari_aditional_data.csv
/kaggle/input/srilanka-geo/Divisional_geo.json


# Interactive choropleth maps with Python and Folium for Srilanka (Grama Niladari Level)

In [11]:
#Let's import libraries we need
import pandas as pd
import geopandas as gpd
import folium

In [12]:
# importing geo json files 
gng=gpd.read_file('/kaggle/input/srilanka-geo/Grama_geo.json')

In [13]:
# loading first two rows of the data set
gng.head(2)

,ADM4_EN,ADM4_SI,ADM4_TA,ADM4_PCODE,ADM4_REF,ADM4ALT1EN,ADM4ALT2EN,ADM4ALT1SI,ADM4ALT2SI,ADM4ALT1TA,...,ADM0_EN,ADM0_SI,ADM0_TA,ADM0_PCODE,Shape_Leng,Shape_Area,date,validOn,validTo,geometry
0,04 1/2 Kanuwa,4 ½ කණුව,,LK7142090,,,,,,,...,Sri Lanka,ශ්‍රී ලංකා,இலங்கை,LK,0.141404,0.00049,2019-03-04T00:00:00,2020-03-05T00:00:00,1899-11-30T00:00:00,"POLYGON ((80.23123 8.18024, 80.22059 8.19033, ..."
1,07 Gammanaya,7 වන ගම්මානය,,LK6224040,,,,,,,...,Sri Lanka,ශ්‍රී ලංකා,இலங்கை,LK,0.122450,0.00034,2019-03-04T00:00:00,2020-03-05T00:00:00,1899-11-30T00:00:00,"POLYGON ((80.02773 7.95962, 80.01903 7.96374, ..."


In [14]:
#removing other data except "ADM4_PCODE" and "geometry"
gng=gng[['ADM4_PCODE','geometry']]

In [15]:
# loading aditional data
gn_a=pd.read_csv('/kaggle/input/srilanka-geo/grama_niladari_aditional_data.csv')
gn_a.rename(columns={'admin4Pcode':'ADM4_PCODE'}, inplace=True)
gn_a.head()

,admin4Name_en,admin4Name_si,ADM4_PCODE,risk_level,patients,population
0,04 1/2 Kanuwa,4 ½ කණුව,LK7142090,1.0,5.0,60.0
1,07 Gammanaya,7 වන ගම්මානය,LK6224040,2.0,10.0,120.0
2,08 Ela,අටේ ඇල,LK2121105,3.0,15.0,180.0
3,18A. Janapadaya,18 A ජනපදය,LK5242140,4.0,20.0,240.0
4,2 Ela - Kagama,කාගම ඇල 2,LK7160115,5.0,25.0,300.0


In [16]:
grama=gng.merge(gn_a,on="ADM4_PCODE")
grama.head()

,ADM4_PCODE,geometry,admin4Name_en,admin4Name_si,risk_level,patients,population
0,LK7142090,"POLYGON ((80.23123 8.18024, 80.22059 8.19033, ...",04 1/2 Kanuwa,4 ½ කණුව,1.000000,5.000000,60.00000
1,LK6224040,"POLYGON ((80.02773 7.95962, 80.01903 7.96374, ...",07 Gammanaya,7 වන ගම්මානය,2.000000,10.000000,120.00000
2,LK2121105,"POLYGON ((80.96122 7.38586, 80.97021 7.39881, ...",08 Ela,අටේ ඇල,3.000000,15.000000,180.00000
3,LK5242140,"POLYGON ((81.73889 7.16753, 81.69939 7.15473, ...",18A. Janapadaya,18 A ජනපදය,4.000000,20.000000,240.00000
4,LK4100000,"POLYGON ((80.25787 9.62729, 80.25045 9.61655, ...",[unknown],[unknown],376.868743,1884.343715,22612.12458


In [17]:
# Defining the style function and Highlight function 

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



# declaring the map with starting point
m = folium.Map(location=[7.8731, 80.7718], zoom_start=8)

choropleth=folium.Choropleth(
    geo_data=grama,
    name='choropleth',
    data= grama,
    columns=['ADM4_PCODE', 'risk_level'],
    key_on='properties.ADM4_PCODE',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    
    legend_name='Covid-19 risk Level with dummy data',

).add_to(m)

# adding some geo loacation (location pins )

folium.Marker(
    location=[6.905655, 79.927357],
    popup='Point one',
    
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[6.7106,79.9074],
    popup='Point two',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[7.9403,81.01886823],
    popup='Point three',
    icon=folium.Icon(color='blue', icon='bar-chart')
).add_to(m)

folium.Marker(
    location=[9.3803, 80.3770],
    popup='Point four',
    icon=folium.Icon(color='red', icon='university')
).add_to(m)

#adding the same geo layer again on top of previouse layer to make interactivity 

INT = folium.features.GeoJson(
    grama,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['admin4Name_en','patients','population'],
        aliases=['GN name: ','Numer of Patients:','Population'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(INT)
m.keep_in_front(INT)
folium.LayerControl().add_to(m)


/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [18]:
m
#save map as html
#m.save('grama_map.html')